In [ ]:
import pandas as pd 


In [1]:
# Convert money lines to probabilites 
def to_prob(money_line):
    if money_line < 0: 
        prob = -money_line / (-money_line + 100) 
    elif money_line > 0: 
        prob = 100 / (money_line + 100) 
    else: 
        prob = None 

    return (prob) 

In [12]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd 

url = 'https://projects.fivethirtyeight.com/2019-mlb-predictions/games/'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

In [42]:
rows = page_content.findAll(class_="tr")

dict_list = []

for row in rows:
    name = row.find(class_='team-name long').getText()
    num_rating = row.find(class_='td number td-number rating-adj').getText().replace('=', '')
    win_prob = row.find(class_='td number td-number win-prob').getText()
    
    try:
        score = row.find(class_='td number td-number score').getText()
    except:
        score = None
    
    d = {
        'name': name,
        'num_rating': num_rating, 
        'win_prob': win_prob,
        'score': score
    }
    
    dict_list.append(d)
    
pd.DataFrame(dict_list)

,name,num_rating,score,win_prob
0,Dodgers,1584,,58%
1,Phillies,1531,,42%
2,Rays,1546,,41%
3,Yankees,1610,,59%
4,Blue Jays,1467,,38%
5,Red Sox,1555,,62%
6,Tigers,1414,,34%
7,Indians,1525,,66%
8,Reds,1530,,44%
9,Cubs,1574,,56%
